In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/atanh'

In [4]:
# !pip install smt

In [5]:
def true_2D_4(xt): #True function for 2D_4 Heat Transfer in a rod x \in [0,1] t \in [0,0.1]
    term1 = 4*u0/np.pi
    
    resol_n = 10000
    
    x = xt[:,0].reshape(-1,1)
    t = xt[:,1].reshape(-1,1)

    u = np.zeros((np.shape(xt)[0],1))
    
    for i in range(resol_n):
        j = 2*i-1
        term2 = np.sin(j*np.pi*x)/j
        term3 = np.exp(-1*np.square(j*np.pi)*t)
        
        u = u + term2*term3
        
    u = term1*u
    
    return u

In [6]:
u0 = 50.0
loss_thresh = 0.1
label = "inv_HT_stan"

x_ll = np.array(0.0)
x_ul = np.array(1.0)

x = np.linspace(x_ll,x_ul,100).reshape(-1,1)
t = np.linspace(0,0.1,100).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

u_true = true_2D_4(xt)
u_true_norm = np.linalg.norm(u_true,2)


lb_xt = xt[0]
ub_xt = xt[-1]

xt_test_tensor = torch.from_numpy(xt).float().to(device)

In [7]:
def trainingdata(N_f,N_train,seed):
    
    np.random.seed(seed)
    
    #X_Train
    np.random.seed(seed)
    x_train = np.random.uniform(x_ll,x_ul,(N_train,1))
    t_train = np.random.uniform(0,0.1,(N_train,1))
    
    xt_train = np.hstack((x_train,t_train))
    u_train = true_2D_4(xt_train)
    

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)

    samples = sampling(N_f)
    
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll)) # append training points to collocation points 

    return xt_coll, xt_train, u_train

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,beta_init):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
        
        
        self.beta = Parameter(beta_init*torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
    
        self.lambda1 = Parameter(torch.tensor(0.0))
        self.lambda1.requiresGrad = True
        

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            z1 = self.activation(z) 
            a = z1 + self.beta[:,i]*z*z1
            
        a = self.linears[-1](a) 
         
        return a
                        
    
    def loss_PDE(self, xt_coll,f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_t = autograd.grad(u,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_tt = autograd.grad(u_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dx = u_x_t[:,[0]]
        
        d2u_dx2 = u_xx_tt[:,[0]]
                
        du_dt = u_x_t[:,[1]]
        
        f = du_dt - self.lambda1*d2u_dx2
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_coll,f_hat, xt_train, u_train):

        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_train = self.loss_function(self.forward(xt_train),u_train)
        
        loss_val = loss_f + loss_train
        
        #print(self.iter,"train_loss",loss_train.cpu().detach().numpy(),"F Loss",(loss_f).cpu().detach().numpy())
        return loss_val
     
    'callable for optimizer'                                    
    
    def test(self):
        u_pred = self.forward(xt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
               
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [9]:
def train_step(xt_coll,f_hat, xt_train, u_train,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_coll,f_hat, xt_train, u_train)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [10]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    lambda1_val.append(PINN.lambda1.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [11]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xt_coll, xt_train, u_train = trainingdata(N_f,N_train,123)
    
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_train = torch.from_numpy(xt_train).float().to(device)
    u_train = torch.from_numpy(u_train).float().to(device)
    
    
    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)

    for i in range(max_iter):
        train_step(xt_coll,f_hat, xt_train, u_train,i)

        loss_np = PINN.loss(xt_coll,f_hat, xt_train, u_train).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1],"Lambda1",lambda1_val[-1])


    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [12]:
max_reps = 10 #10
max_iter = 200 #75

train_loss_full = []
test_mse_full = []
test_re_full = []
beta_full = []

lambda1_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

beta_init = 0.5

for reps in range(max_reps):
    print(label)
    'Generate Training data'
    print(reps)
    torch.manual_seed(reps*36)

    train_loss = []
    test_mse_loss = []
    test_re_loss = []   
    beta_val = []

    lambda1_val = []

    N_f = 50000 #Total number of collocation points 
    N_train = 5000

    layers = np.array([2,50,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

    PINN = Sequentialmodel(layers,beta_init)

    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())

    optimizer = torch.optim.LBFGS(PINN.parameters(), lr= 0.5, 
                              max_iter = 20, 
                              max_eval = 30, 
                              tolerance_grad = 1e-8, 
                              tolerance_change = 1e-8, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')


    nan_flag = train_model(max_iter,reps)

    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time
    beta_full.append(beta_val)

    lambda1_full.append(lambda1_val)


    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full,"lambda1": lambda1_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic) 


inv_HT_stan
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 853.5218 Test MSE 856.0882215965379 Test RE 0.49254785441163884 Lambda1 -0.020152098
1 Train Loss 764.9929 Test MSE 768.1692004433863 Test RE 0.4665708649500331 Lambda1 0.008244848
2 Train Loss 632.5328 Test MSE 

77 Train Loss 31.488708 Test MSE 34.54180708493354 Test RE 0.09893767441002027 Lambda1 1.1907473
78 Train Loss 31.437145 Test MSE 34.40666098426087 Test RE 0.09874393606282461 Lambda1 1.1846766
79 Train Loss 31.394976 Test MSE 34.31038093816427 Test RE 0.098605681860487 Lambda1 1.1813765
80 Train Loss 31.333452 Test MSE 34.1587296911521 Test RE 0.09838752288692172 Lambda1 1.1703073
81 Train Loss 31.264444 Test MSE 34.01214962012362 Test RE 0.09817619836085362 Lambda1 1.1603189
82 Train Loss 31.170023 Test MSE 33.9198489968767 Test RE 0.09804289481230213 Lambda1 1.1467794
83 Train Loss 31.046858 Test MSE 33.85027939004651 Test RE 0.09794230022232783 Lambda1 1.1438252
84 Train Loss 30.937456 Test MSE 33.85313978348691 Test RE 0.09794643826195494 Lambda1 1.1427876
85 Train Loss 30.881643 Test MSE 33.86596372289691 Test RE 0.09796498809615074 Lambda1 1.1451486
86 Train Loss 30.81854 Test MSE 33.77845385909801 Test RE 0.09783833515175094 Lambda1 1.1433234
87 Train Loss 30.70519 Test MSE 33.

161 Train Loss 27.115973 Test MSE 30.30602014799491 Test RE 0.09267308583778189 Lambda1 0.9910456
162 Train Loss 27.093533 Test MSE 30.2659705450524 Test RE 0.09261183155095092 Lambda1 0.9915842
163 Train Loss 27.035936 Test MSE 30.233407891820946 Test RE 0.09256199838072594 Lambda1 0.9879446
164 Train Loss 26.985628 Test MSE 30.159520691529945 Test RE 0.09244882340245382 Lambda1 0.98937106
165 Train Loss 26.94237 Test MSE 30.175982265951916 Test RE 0.09247405002318836 Lambda1 0.9884902
166 Train Loss 26.933655 Test MSE 30.16347004302062 Test RE 0.09245487623317127 Lambda1 0.98699623
167 Train Loss 26.926556 Test MSE 30.17225560619496 Test RE 0.09246833968775812 Lambda1 0.986599
168 Train Loss 26.912912 Test MSE 30.14583165620356 Test RE 0.09242784032951812 Lambda1 0.98657227
169 Train Loss 26.893147 Test MSE 30.110537811730815 Test RE 0.09237371859851992 Lambda1 0.98121697
170 Train Loss 26.863592 Test MSE 30.06036574375271 Test RE 0.09229672706925435 Lambda1 0.9756916
171 Train Loss 

36 Train Loss 96.79539 Test MSE 89.74241868152322 Test RE 0.1594733273556439 Lambda1 0.7806508
37 Train Loss 92.92994 Test MSE 87.37560332452281 Test RE 0.15735634679986849 Lambda1 0.80123496
38 Train Loss 89.673195 Test MSE 83.32124318353003 Test RE 0.15366219804272507 Lambda1 0.8177233
39 Train Loss 85.298546 Test MSE 79.88547146435836 Test RE 0.15046069790747266 Lambda1 0.83541375
40 Train Loss 81.097084 Test MSE 75.10829916772686 Test RE 0.14589255643084256 Lambda1 0.87394035
41 Train Loss 78.12879 Test MSE 73.58518420164117 Test RE 0.14440570838353325 Lambda1 0.8897321
42 Train Loss 73.73964 Test MSE 68.50383947725669 Test RE 0.1393306384940783 Lambda1 0.9335494
43 Train Loss 69.342026 Test MSE 63.59012644512902 Test RE 0.1342406402765859 Lambda1 0.9864053
44 Train Loss 67.194725 Test MSE 63.18355322677295 Test RE 0.13381080807263426 Lambda1 0.9899833
45 Train Loss 63.922356 Test MSE 60.337821718586575 Test RE 0.130762731619327 Lambda1 1.0407912
46 Train Loss 62.802055 Test MSE 58

121 Train Loss 30.741447 Test MSE 33.16229072308844 Test RE 0.09694187866382332 Lambda1 1.3031678
122 Train Loss 30.618544 Test MSE 33.10371784260395 Test RE 0.09685622905353057 Lambda1 1.2999494
123 Train Loss 30.511875 Test MSE 33.00567681557088 Test RE 0.09671269645282385 Lambda1 1.2954569
124 Train Loss 30.465982 Test MSE 32.98790483354472 Test RE 0.09668665536079091 Lambda1 1.2924812
125 Train Loss 30.402527 Test MSE 32.892882028395 Test RE 0.09654730030037485 Lambda1 1.2896543
126 Train Loss 30.313667 Test MSE 32.83337649865577 Test RE 0.09645993035026497 Lambda1 1.2868564
127 Train Loss 30.246109 Test MSE 32.7666650174071 Test RE 0.09636188595874097 Lambda1 1.2827632
128 Train Loss 30.090582 Test MSE 32.634455542664654 Test RE 0.09616728526698486 Lambda1 1.2747065
129 Train Loss 29.901697 Test MSE 32.480680473599264 Test RE 0.09594044537580282 Lambda1 1.2623174
130 Train Loss 29.691942 Test MSE 32.4112528106858 Test RE 0.09583785386519271 Lambda1 1.2578385
131 Train Loss 29.6215

...

16 Train Loss 78.85812 Test MSE 70.152165659709 Test RE 0.14099694795642795 Lambda1 0.8480421
17 Train Loss 75.175896 Test MSE 66.81889705549777 Test RE 0.13760645982259603 Lambda1 0.8779568
18 Train Loss 71.087814 Test MSE 64.4444901724405 Test RE 0.13513942501458237 Lambda1 0.8786881
19 Train Loss 67.98253 Test MSE 60.07933412746819 Test RE 0.13048233682943267 Lambda1 0.9049527
20 Train Loss 62.180206 Test MSE 57.08807672081674 Test RE 0.1271926092602992 Lambda1 0.9604816
21 Train Loss 60.08065 Test MSE 55.543888793439095 Test RE 0.12546058608481073 Lambda1 0.9716917
22 Train Loss 57.16109 Test MSE 53.51225056973577 Test RE 0.12314471526316212 Lambda1 0.9965424
23 Train Loss 55.463993 Test MSE 52.201697245113785 Test RE 0.12162741662747821 Lambda1 0.9968163
24 Train Loss 53.286446 Test MSE 51.29800196625437 Test RE 0.12057003738329668 Lambda1 1.0119094
25 Train Loss 49.661278 Test MSE 47.364741799170126 Test RE 0.1158555271749819 Lambda1 1.0547546
26 Train Loss 46.368183 Test MSE 44.

101 Train Loss 29.092958 Test MSE 32.142554603970616 Test RE 0.09543976599001287 Lambda1 1.0892309
102 Train Loss 29.055206 Test MSE 32.092624179142284 Test RE 0.09536560884555857 Lambda1 1.0731002
103 Train Loss 29.038334 Test MSE 32.07984165163713 Test RE 0.09534661484059509 Lambda1 1.0705818
104 Train Loss 28.996239 Test MSE 32.03341618101224 Test RE 0.09527759775862425 Lambda1 1.0602585
105 Train Loss 28.970634 Test MSE 32.01380022233794 Test RE 0.0952484212322081 Lambda1 1.0524716
106 Train Loss 28.94973 Test MSE 31.985756965271406 Test RE 0.09520669452141674 Lambda1 1.0403985
107 Train Loss 28.932192 Test MSE 31.976392675824055 Test RE 0.09519275693806749 Lambda1 1.0381137
108 Train Loss 28.90613 Test MSE 31.968256463164384 Test RE 0.09518064553770475 Lambda1 1.0348632
109 Train Loss 28.876152 Test MSE 31.952630871063203 Test RE 0.09515738127670327 Lambda1 1.0340947
110 Train Loss 28.839748 Test MSE 31.932743434876215 Test RE 0.09512776351358347 Lambda1 1.0365891
111 Train Loss 2

185 Train Loss 28.018822 Test MSE 31.524791513235197 Test RE 0.09451816513832507 Lambda1 0.9629169
186 Train Loss 28.018822 Test MSE 31.524791513235197 Test RE 0.09451816513832507 Lambda1 0.9629169
187 Train Loss 28.018822 Test MSE 31.524791513235197 Test RE 0.09451816513832507 Lambda1 0.9629169
188 Train Loss 28.018822 Test MSE 31.524791513235197 Test RE 0.09451816513832507 Lambda1 0.9629169
189 Train Loss 28.018822 Test MSE 31.524791513235197 Test RE 0.09451816513832507 Lambda1 0.9629169
190 Train Loss 28.018822 Test MSE 31.524791513235197 Test RE 0.09451816513832507 Lambda1 0.9629169
191 Train Loss 28.018822 Test MSE 31.524791513235197 Test RE 0.09451816513832507 Lambda1 0.9629169
192 Train Loss 28.018822 Test MSE 31.524791513235197 Test RE 0.09451816513832507 Lambda1 0.9629169
193 Train Loss 28.018822 Test MSE 31.524791513235197 Test RE 0.09451816513832507 Lambda1 0.9629169
194 Train Loss 28.018822 Test MSE 31.524791513235197 Test RE 0.09451816513832507 Lambda1 0.9629169
195 Train 

61 Train Loss 30.22435 Test MSE 30.741099012012665 Test RE 0.09333593136347014 Lambda1 1.2108686
62 Train Loss 29.99869 Test MSE 30.664352009731413 Test RE 0.0932193491717107 Lambda1 1.219795
63 Train Loss 29.535173 Test MSE 30.6514101610117 Test RE 0.09319967554491543 Lambda1 1.2253789
64 Train Loss 29.192097 Test MSE 30.374512523359257 Test RE 0.09277774850011901 Lambda1 1.2408155
65 Train Loss 28.985783 Test MSE 30.3576580110467 Test RE 0.09275200420731629 Lambda1 1.2499037
66 Train Loss 28.876522 Test MSE 30.341625608768496 Test RE 0.09272750900700465 Lambda1 1.2508084
67 Train Loss 28.734058 Test MSE 30.29689563935026 Test RE 0.09265913382385613 Lambda1 1.2574664
68 Train Loss 28.567966 Test MSE 30.295464073816465 Test RE 0.09265694466905075 Lambda1 1.2522889
69 Train Loss 28.471191 Test MSE 30.289935823856112 Test RE 0.09264849036514468 Lambda1 1.2495896
70 Train Loss 28.420841 Test MSE 30.265352132470625 Test RE 0.09261088539569172 Lambda1 1.2493453
71 Train Loss 28.347445 Test 

145 Train Loss 24.524166 Test MSE 27.27972906068657 Test RE 0.08792435618862854 Lambda1 0.9732682
146 Train Loss 24.487959 Test MSE 27.236675425085128 Test RE 0.08785494644587963 Lambda1 0.9725433
147 Train Loss 24.465658 Test MSE 27.20670080714276 Test RE 0.08780658989451949 Lambda1 0.9692893
148 Train Loss 24.446379 Test MSE 27.184025913377084 Test RE 0.08776999192319719 Lambda1 0.9654478
149 Train Loss 24.431944 Test MSE 27.178078224650967 Test RE 0.08776038964518361 Lambda1 0.96476597
150 Train Loss 24.41667 Test MSE 27.14612282455665 Test RE 0.08770878108093855 Lambda1 0.96135914
151 Train Loss 24.399351 Test MSE 27.130893276857304 Test RE 0.08768417439079673 Lambda1 0.95916057
152 Train Loss 24.385336 Test MSE 27.134570546951913 Test RE 0.08769011646325328 Lambda1 0.9569889
153 Train Loss 24.369576 Test MSE 27.095590977604175 Test RE 0.0876271091750103 Lambda1 0.95311147
154 Train Loss 24.350689 Test MSE 27.096885976215905 Test RE 0.08762920316198013 Lambda1 0.95093703
155 Train 